In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser
from datetime import datetime
import requests
from pandas import json_normalize
import os
from dotenv import load_dotenv
load_dotenv()
import meteomatics.api as api
import datetime as dt
from bs4 import BeautifulSoup
import re
#import src.manage_data as dat
import json
#from sklearn import preprocessign

In [2]:
planes = pd.read_csv("planes.csv")

In [79]:
display(planes.tail(5))
df = planes.copy()

#df["tweet_coord"].unique()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,2015-02-22 11:58:51 -0800,"dallas, TX",NaN


In [80]:
display(df.dtypes)

tweet_id                          int64
airline_sentiment                object
airline_sentiment_confidence    float64
negativereason                   object
negativereason_confidence       float64
airline                          object
airline_sentiment_gold           object
name                             object
negativereason_gold              object
retweet_count                     int64
text                             object
tweet_coord                      object
tweet_created                    object
tweet_location                   object
user_timezone                    object
dtype: object

In [81]:
#lets convert tweet_created from object to DATE TYPE
df["tweet_created"] = pd.to_datetime(df["tweet_created"])
display(df.dtypes)

tweet_id                                                         int64
airline_sentiment                                               object
airline_sentiment_confidence                                   float64
negativereason                                                  object
negativereason_confidence                                      float64
airline                                                         object
airline_sentiment_gold                                          object
name                                                            object
negativereason_gold                                             object
retweet_count                                                    int64
text                                                            object
tweet_coord                                                     object
tweet_created                   datetime64[ns, pytz.FixedOffset(-480)]
tweet_location                                                  object
user_t

In [82]:
#Deleting Tweet ID
df.drop("tweet_id", axis = 1, inplace = True)
display(df.dtypes)

airline_sentiment                                               object
airline_sentiment_confidence                                   float64
negativereason                                                  object
negativereason_confidence                                      float64
airline                                                         object
airline_sentiment_gold                                          object
name                                                            object
negativereason_gold                                             object
retweet_count                                                    int64
text                                                            object
tweet_coord                                                     object
tweet_created                   datetime64[ns, pytz.FixedOffset(-480)]
tweet_location                                                  object
user_timezone                                                   object
dtype:

In [83]:
#setting index based on date of creation

In [84]:
df.columns

Index(['airline_sentiment', 'airline_sentiment_confidence', 'negativereason',
       'negativereason_confidence', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_location', 'user_timezone'],
      dtype='object')

In [85]:
df.set_index("tweet_created", inplace = True)

In [86]:
df.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone
tweet_created,,,,,,,,,,,,,
2015-02-24 11:35:52-08:00,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,NaN,Eastern Time (US & Canada)
2015-02-24 11:15:59-08:00,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,NaN,Pacific Time (US & Canada)
2015-02-24 11:15:48-08:00,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,Lets Play,Central Time (US & Canada)
2015-02-24 11:15:36-08:00,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,NaN,Pacific Time (US & Canada)
2015-02-24 11:14:45-08:00,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,NaN,Pacific Time (US & Canada)


In [87]:
df.describe()

,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,14640.000000,10522.000000,14640.000000
mean,0.900169,0.638298,0.082650
std,0.162830,0.330440,0.745778
min,0.335000,0.000000,0.000000
25%,0.692300,0.360600,0.000000
50%,1.000000,0.670600,0.000000
75%,1.000000,1.000000,0.000000
max,1.000000,1.000000,44.000000


In [88]:
#I would like to get a mean of ratings. Let's suppose that our customers can rate between 1 and 3 
#beeing 1 negative and 3 positive. 
#To achive this I will create a new column base on this condition

conditions = [(df["airline_sentiment"] == "negative"),
             (df["airline_sentiment"] == "neutral"),
             (df["airline_sentiment"] == "positive")]
values = [1, 2, 3]
df["sentiment_rate"] = np.select(conditions, values)


#df.columns = ['rating_sentiment','airline_sentiment','sentiment_rate', 'airline_sentiment_confidence', 'negativereason',
#       'negativereason_confidence', 'airline', 'airline_sentiment_gold',
#       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
#       'tweet_location', 'user_timezone']

#df

#Esto se hace con un un diccionario y un map
#Sentimientos a lo largo del tiempo


In [89]:
df.reset_index(inplace=True)
#df = df.rename(columns = {'index':'new column name'})
df

,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate
0,2015-02-24 11:35:52-08:00,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,NaN,Eastern Time (US & Canada),2
1,2015-02-24 11:15:59-08:00,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,NaN,Pacific Time (US & Canada),3
2,2015-02-24 11:15:48-08:00,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,Lets Play,Central Time (US & Canada),2
3,2015-02-24 11:15:36-08:00,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,NaN,Pacific Time (US & Canada),1
4,2015-02-24 11:14:45-08:00,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,NaN,Pacific Time (US & Canada),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,2015-02-22 12:01:01-08:00,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,NaN,NaN,3
14636,2015-02-22 11:59:46-08:00,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,Texas,NaN,1
14637,2015-02-22 11:59:15-08:00,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,"Nigeria,lagos",NaN,2
14638,2015-02-22 11:59:02-08:00,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,New Jersey,Eastern Time (US & Canada),1


In [90]:
df.tweet_coord.count() #dropna twittcord

1019

In [92]:
df['tweet_created'] = df.tweet_created.astype(str)
df["datetime"] = df.tweet_created.apply(lambda x: dateutil.parser.parse(x))
df["date"] = df.datetime.apply(lambda x: x.date())
df.head()

,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate,datetime,date
0,2015-02-24 11:35:52-08:00,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,NaN,Eastern Time (US & Canada),2,2015-02-24 11:35:52-08:00,2015-02-24
1,2015-02-24 11:15:59-08:00,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,NaN,Pacific Time (US & Canada),3,2015-02-24 11:15:59-08:00,2015-02-24
2,2015-02-24 11:15:48-08:00,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,Lets Play,Central Time (US & Canada),2,2015-02-24 11:15:48-08:00,2015-02-24
3,2015-02-24 11:15:36-08:00,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,NaN,Pacific Time (US & Canada),1,2015-02-24 11:15:36-08:00,2015-02-24
4,2015-02-24 11:14:45-08:00,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,NaN,Pacific Time (US & Canada),1,2015-02-24 11:14:45-08:00,2015-02-24


In [94]:

df.head()

,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate,datetime,date
0,2015-02-24 11:35:52-08:00,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,NaN,Eastern Time (US & Canada),2,2015-02-24 11:35:52-08:00,2015-02-24
1,2015-02-24 11:15:59-08:00,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,NaN,Pacific Time (US & Canada),3,2015-02-24 11:15:59-08:00,2015-02-24
2,2015-02-24 11:15:48-08:00,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,Lets Play,Central Time (US & Canada),2,2015-02-24 11:15:48-08:00,2015-02-24
3,2015-02-24 11:15:36-08:00,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,NaN,Pacific Time (US & Canada),1,2015-02-24 11:15:36-08:00,2015-02-24
4,2015-02-24 11:14:45-08:00,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,NaN,Pacific Time (US & Canada),1,2015-02-24 11:14:45-08:00,2015-02-24


In [95]:
df.date.unique()

array([datetime.date(2015, 2, 24), datetime.date(2015, 2, 23),
       datetime.date(2015, 2, 22), datetime.date(2015, 2, 21),
       datetime.date(2015, 2, 20), datetime.date(2015, 2, 19),
       datetime.date(2015, 2, 18), datetime.date(2015, 2, 17),
       datetime.date(2015, 2, 16)], dtype=object)

In [96]:
df.negativereason.unique()

array([nan, 'Bad Flight', "Can't Tell", 'Late Flight',
       'Customer Service Issue', 'Flight Booking Problems',
       'Lost Luggage', 'Flight Attendant Complaints', 'Cancelled Flight',
       'Damaged Luggage', 'longlines'], dtype=object)

In [97]:
df.airline.unique()
#- Do tweets with negative raiting get more retweets? 


array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

In [101]:
#FAILED TRYING OPEN WATHER
#params = {"Auhorization" : f"token {api_key}"}
#response = requests.get( url = url , params = params).json()

#https://openweathermap.org/history

#response = requests.get("http://history.openweathermap.org/data/2.5/history/city?q=London,UK&appid=4b8ef34fa263f86daf9ae3d165932f59").json()

#response

In [102]:

username = 'imprexeurope_campos'
password = 'b4fFJ02cTuCRp'

In [103]:
now = dt.datetime.utcnow().replace(2015, 2, 24)
startdate_ts = now
enddate_ts = startdate_ts + dt.timedelta(days=1)
interval_ts = dt.timedelta(days=1)

In [104]:
coordinates_ts = [(47.25,9.34), (30,-4)]

In [105]:
parameters_ts = ['t_2m:C']

In [106]:
print("time series:")
try:
    df_ts = api.query_time_series(coordinates_ts, startdate_ts, enddate_ts, interval_ts,
                                  parameters_ts, username, password)
    print (df_ts.head())
except Exception as e:
    print("Failed, the exception is {}".format(e))

time series:
Failed, the exception is Request with valid date 2015-02-24 19:07:07 requires data access before 2021-05-27 00:00:00, which is not granted with this subscription type (e.g. trial). The valid time period for this account type starts at 2021-05-27 00:00:00 and ends at 2023-05-28 00:00:00. Please contact sales@meteomatics.com and we are happy to provide an extended trial or an upgrade of your account.


In [107]:
url = "https://www.airlineratings.com/airline-ratings/?l=A"
html = requests.get(url)
soup = BeautifulSoup(html.content,"html.parser")

In [108]:
rating = soup.find_all("div", {"class": "rating"})
#    rating[0]["style"][6:-1]


nombres = soup.select("h2",{"ul":"airlines"})
lista_nombres= [a.getText().strip() for a in nombres[1:-1]]    
rating_list = [float(r["style"][6:-1]) for r in rating]
#rating_list = [int(e) for e in rating_list]

#rating_list

rating[0]["style"]
#rating_list[0]

'width:100%'

In [109]:
type(rating[0])
type(rating[0]["style"])
 #string
#rating[0]

str

In [110]:
#import funciones

#Failed importing funciones
#failing imporing SRC module

def urlf(letter):
    url =f"https://www.airlineratings.com/airline-ratings/?l={letter}" 
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    return soup



In [114]:
import funciones

#D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","W","X","Y","Z"
alphabet = ["A", "B", "C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","W","X","Y","Z"]
sites = []
for letter in alphabet:
    sites.append(urlf(letter))


    
#url = "https://www.airlineratings.com/airline-ratings/?l=A"
#un dateframe es una lista de muchos diccionarios -pillalo con pinzas {key:value}



NameError: name 'urlf' is not defined

In [34]:
type(sites)

list

In [35]:
sites[0]

<!DOCTYPE html >
 <!--[if IE 8]><html class=ie8 lang=en> <![endif]--> <!--[if IE 9]><html class=ie9 lang=en> <![endif]--> <!--[if gt IE 8]><!--><html lang="en-AU" prefix="og: http://ogp.me/ns#"> <!--<![endif]--><head><title>Airline &amp; Safety Ratings - Airline Ratings</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><link href="" rel="pingback"/><link href="https://www.airlineratings.com/airline-ratings/" rel="canonical"/><meta content="en_US" property="og:locale"/><meta content="article" property="og:type"/><meta content="Airline &amp; Safety Ratings - Airline Ratings" property="og:title"/><meta content="More information on our airline and safety ratings can be found here or by simply clicking on the safety rating link next to each airline. The maximum rating an airline can get for safety is seven stars. We also provide information on the history, aircraft types and in-flight offering (meals, drinks, seating and in-flight entertain

In [36]:
airlines_names = []
airlines_reviews = []

for site in sites:
    nombres = site.select("h2",{"ul":"airlines"})
    lista_nombres= [a.getText().strip() for a in nombres[1:-1]]
    airlines_names.append(lista_nombres)
    rating = site.find_all("div", {"class": "rating"})
    rating_list = [float(r["style"][6:-1]) for r in rating]
    rating_list = [int(e) for e in rating_list]
    airlines_reviews.append(rating_list)

airlines_reviews

[[100,
  100,
  85,
  71,
  42,
  71,
  100,
  100,
  100,
  28,
  14,
  100,
  85,
  100,
  100,
  85,
  71,
  85,
  71,
  85,
  100,
  100,
  100,
  85,
  100,
  100,
  100,
  85,
  85,
  71,
  42,
  85,
  71,
  71,
  100,
  85,
  100,
  71,
  92,
  85,
  85,
  100,
  85,
  100,
  57,
  85,
  100,
  71,
  100,
  57,
  85,
  85,
  85,
  100,
  71,
  100,
  85,
  71,
  71,
  42,
  85,
  85,
  71,
  85,
  85,
  14,
  85,
  42,
  100,
  85,
  100,
  100,
  100,
  100,
  85,
  100,
  100,
  71,
  85,
  71,
  85,
  85,
  57,
  71,
  42,
  85,
  85,
  71,
  100,
  71,
  100,
  85,
  100,
  71,
  85,
  100,
  85,
  22,
  29,
  53,
  58,
  69],
 [100,
  100,
  100,
  100,
  85,
  57,
  85,
  100,
  85,
  100,
  85,
  100,
  14,
  100,
  100,
  85,
  22,
  29,
  53,
  58,
  69],
 [71,
  85,
  85,
  100,
  100,
  100,
  85,
  100,
  100,
  85,
  100,
  85,
  85,
  100,
  71,
  71,
  100,
  100,
  28,
  100,
  85,
  100,
  100,
  42,
  100,
  22,
  29,
  53,
  58,
  69],
 [100, 100, 85, 22, 29, 

In [37]:
rating[0]["style"]

'width:22%'

In [38]:

flat_airline = []
for sublist in airlines_names:
    for item in sublist:
        flat_airline.append(item)

        


In [39]:
flat_reviews = []
for sublist in airlines_reviews:
    for item in sublist:
        flat_reviews.append(item)


In [40]:
print(len(flat_reviews))
print(len(flat_airline))


459
459


In [41]:
reviews_by_airline = {
    
    "flat_reviews": flat_reviews,
    "flat_airline" : flat_airline
}

data = pd.DataFrame.from_dict(reviews_by_airline)
data

,flat_reviews,flat_airline
0,100,Aegean Airlines
1,100,Aer Lingus
2,85,Aero Contractors
3,71,Aero Mongolia
4,42,Aeroflot
...,...,...
454,22,American Airlines
455,29,United Airlines
456,53,Emirates
457,58,Qantas


### data.drop_duplicates(subset="flat_airline", inplace= True)
data

In [42]:
data.flat_airline.unique()

array(['Aegean Airlines', 'Aer Lingus', 'Aero Contractors',
       'Aero Mongolia', 'Aeroflot', 'Aerogaviota',
       'Aerolineas Argentinas', 'Aeromar', 'Aeromexico', 'Afriqiyah',
       'Air Algerie', 'Air Arabia', 'Air Astana', 'Air Austral',
       'Air Baltic', 'Air Botswana', 'Air Busan', 'Air Cairo',
       'Air Canada', 'Air Canada Express', 'Air Caraibes', 'Air China',
       'Air Corsica', 'Air Do', 'Air Dolomiti', 'Air Europa',
       'Air France', 'Air Greenland', 'Air Iceland', 'Air India',
       'Air India Express', 'Air Inuit', 'Air KBZ', 'Air Koryo',
       'Air Macau', 'Air Madagascar', 'Air Malta', 'Air Mandalay',
       'Air Mauritius', 'Air Mediterranee', 'Air Moldova', 'Air Namibia',
       'Air Nauru', 'Air New Zealand', 'Air Niugini',
       'Air North Yukons Airline', 'Air Nostrum/Iberia Regional',
       'Air Panama', 'Air Peace', 'Air Rarotonga', 'Air Serbia',
       'Air Seychelles', 'Air Tahiti', 'Air Tahiti Nui', 'Air Tanzania',
       'Air Transat', 'Air 

In [43]:
data.head()

,flat_reviews,flat_airline
0,100,Aegean Airlines
1,100,Aer Lingus
2,85,Aero Contractors
3,71,Aero Mongolia
4,42,Aeroflot


In [44]:
df.head(3)

,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate,datetime,date
0,2015-02-24 11:35:52-08:00,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,NaN,Eastern Time (US & Canada),2,2015-02-24 11:35:52-08:00,2015-02-24
1,2015-02-24 11:15:59-08:00,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,NaN,Pacific Time (US & Canada),3,2015-02-24 11:15:59-08:00,2015-02-24
2,2015-02-24 11:15:48-08:00,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,Lets Play,Central Time (US & Canada),2,2015-02-24 11:15:48-08:00,2015-02-24


In [45]:

data.flat_airline.replace(to_replace =["American Airlines"], 
                         value = "American", inplace = True)

data.flat_airline.replace(to_replace =["United Airlines"], 
                         value = "United", inplace = True)

data.flat_airline.replace(to_replace =["Virgin Atlantic"], 
                         value = "Virgin America", inplace = True)


data.flat_airline.replace(to_replace =["Southwest Airlines"], 
                         value = "Southwest", inplace = True)

data.flat_airline.replace(to_replace =["Delta Air Lines"], 
                         value = "Delta", inplace = True)


#crear un diccionario y usar un apply, map, etc
#apply con lambda


In [46]:
df_merge = df.merge(data, left_on='airline', right_on='flat_airline')


In [64]:
df_merge[df_merge["flat_airline"] == "United"]
df_merge[df_merge["flat_airline"]== "Southwest"]
df_merge[df_merge["flat_airline"]== "Delta"]
df_merge[df_merge["flat_airline"]== "American"]

,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate,datetime,date,flat_reviews,flat_airline
96370,2015-02-24 11:44:31-08:00,negative,1.0000,Customer Service Issue,1.0,American,NaN,SweeLoTmac,NaN,0,@AmericanAir why would I even consider continu...,NaN,NaN,Quito,1,2015-02-24 11:44:31-08:00,2015-02-24,100,American
96371,2015-02-24 11:44:31-08:00,negative,1.0000,Customer Service Issue,1.0,American,NaN,SweeLoTmac,NaN,0,@AmericanAir why would I even consider continu...,NaN,NaN,Quito,1,2015-02-24 11:44:31-08:00,2015-02-24,22,American
96372,2015-02-24 11:44:31-08:00,negative,1.0000,Customer Service Issue,1.0,American,NaN,SweeLoTmac,NaN,0,@AmericanAir why would I even consider continu...,NaN,NaN,Quito,1,2015-02-24 11:44:31-08:00,2015-02-24,22,American
96373,2015-02-24 11:44:31-08:00,negative,1.0000,Customer Service Issue,1.0,American,NaN,SweeLoTmac,NaN,0,@AmericanAir why would I even consider continu...,NaN,NaN,Quito,1,2015-02-24 11:44:31-08:00,2015-02-24,22,American
96374,2015-02-24 11:44:31-08:00,negative,1.0000,Customer Service Issue,1.0,American,NaN,SweeLoTmac,NaN,0,@AmericanAir why would I even consider continu...,NaN,NaN,Quito,1,2015-02-24 11:44:31-08:00,2015-02-24,22,American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165340,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165341,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165342,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165343,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American


In [47]:
df_merge.drop([96370])


,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate,datetime,date,flat_reviews,flat_airline
0,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
1,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
2,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
3,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
4,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165340,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165341,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165342,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165343,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American


In [48]:
df_merge

,tweet_created,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_location,user_timezone,sentiment_rate,datetime,date,flat_reviews,flat_airline
0,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
1,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
2,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
3,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
4,2015-02-24 11:42:48-08:00,positive,1.0000,NaN,NaN,United,NaN,rdowning76,NaN,0,@united thanks,NaN,usa,NaN,3,2015-02-24 11:42:48-08:00,2015-02-24,29,United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165340,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165341,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165342,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American
165343,2015-02-22 11:58:51-08:00,neutral,0.6771,NaN,0.0,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,NaN,"dallas, TX",NaN,2,2015-02-22 11:58:51-08:00,2015-02-22,22,American


In [50]:
df_merge.to_csv('planes_clean.csv')
